In [37]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"


In [38]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit
np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="30m",
    limit=200
)
pd.DataFrame(candles)

It took 00 mins and 00 seconds to download 200 candles


,0,1,2,3,4
0,1697504400000.00,28263.80,28343.80,28254.00,28341.20
1,1697506200000.00,28341.20,28408.30,28314.40,28337.20
2,1697508000000.00,28337.20,28350.10,28277.00,28327.80
3,1697509800000.00,28327.80,28378.70,28313.80,28358.10
4,1697511600000.00,28358.10,28359.30,28264.80,28267.70
...,...,...,...,...,...
195,1697855400000.00,29507.90,29507.90,29445.10,29471.90
196,1697857200000.00,29471.90,29538.30,29446.40,29530.90
197,1697859000000.00,29530.90,29614.60,29530.90,29591.20
198,1697860800000.00,29591.20,29617.80,29571.70,29575.10


In [84]:
@njit()
def try_func():
    try:
        raise ValueError("You can't be doing that")
    except:
        print('yes')
try_func()

yes


In [29]:
for i in range(5000):
    closing_p = candles[:, 4]
    up = np.zeros(closing_p.size)
    down = np.zeros(closing_p.size)
    mean_up = np.zeros(closing_p.size)
    mean_down = np.zeros(closing_p.size)
    rsi = np.full(closing_p.size, np.nan)
    for counter in range(1, 13):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)

    for counter in range(14, 200):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)
        mean_up[counter] = np.mean(up[counter - 14 : counter + 1])
        mean_down[counter] = np.mean(down[counter - 14 : counter + 1])
        rsi[counter] = 100 - 100 / (1 + (mean_up[counter] / mean_down[counter]))

In [35]:
@njit(cache=True)
def numba_test():
    closing_p = candles[:, 4]
    up = np.zeros(closing_p.size)
    down = np.zeros(closing_p.size)
    mean_up = np.zeros(closing_p.size)
    mean_down = np.zeros(closing_p.size)
    rsi = np.full(closing_p.size, np.nan)
    for counter in range(1, 13):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)

    for counter in range(14, 200):
        pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
        if pchg > 0:
            up[counter] = pchg
        else:
            down[counter] = abs(pchg)
        mean_up[counter] = np.mean(up[counter - 14 : counter + 1])
        mean_down[counter] = np.mean(down[counter - 14 : counter + 1])
        rsi[counter] = 100 - 100 / (1 + (mean_up[counter] / mean_down[counter]))
for i in range(500000):
    numba_test()

In [36]:
from nb_quantfreedom.indicators.indicators import IndicatorsNB
from nb_quantfreedom.nb_enums import CandleBodyType


@njit(cache=True)
def tester():
    return IndicatorsNB().calc_rsi(prices=candles[:, CandleBodyType.Close], period=14)

for i in range(500000):
    tester()